In [8]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs
import requests
import datetime, time
from datetime import date

import pandas as pd
import re
import os, sys

In [9]:
# 삼성전자 005930, 현대차 005380, 네이버 035420, KT&G 033780, 
# code = {'sec': '005930', 'hyunmotor': '005380', 'naver': '035420', 'ktng': '033780'}
code = {'sec': 'KR7005930003', 'hyunmotor': 'KR7005380001', 'naver': 'KR7035420009', 'ktng': 'KR7033780008'}
# code = {'sec': 'KR7005930003'}

In [10]:
today_o = datetime.date.today()
today = today_o.strftime('%Y%m%d')
today_stock = today_o.strftime('%Y-%m-%d')

In [11]:
def get_data(ticker, today):
    url_base = 'https://stock.mk.co.kr/price/hourly?stock_code='
#     url_base = 'https://stock.mk.co.kr/price/hourly?stock_code=KR7005930003&day=2023-01-25&page=2'
    for i in range(8):
        str1 = str(i+1)
        url = '{}{}{}{}{}{}'.format(url_base, ticker, '&day=', today, '&page=', str1)
        print("*********url***", url)
        df1 = pd.read_html(url, attrs={"class": "table-stock line"}, flavor=["lxml", "bs4"])[0]
        print(df1.head())
        if i == 0 :
            df = df1
        else:
#             df = df.append(df1) # append will be depreciated
            df=pd.concat([df,df1], ignore_index=True)
    return df

In [12]:
column_eng = ['date', 'price', 'change', 'change(%)', 'amount', 'volume']
column_kor = ['날짜', '현재가', '등락', '등락률(%)', '체결량', '거래량']

In [13]:
dir_name = 'data/company_daily_price_pkl/'
f_name = '_10min_price_'

In [14]:
for key, val in code.items():
    df = get_data(val, today_stock)
    df.columns = column_eng
    df_temp = df['date']
    yr = today[:4]
    df['date'] = df_temp.apply(lambda x : datetime.datetime.strptime(yr+x[:5], "%Y%m/%d"))
    df.insert(1, 'time', df_temp.apply(lambda x : datetime.datetime.strptime(x[-5:], "%H:%M").time()))
    filename = '{}{}{}{}.pkl'.format(dir_name, key, f_name, today) # code만 기입
    df.to_pickle(filename)

*********url*** https://stock.mk.co.kr/price/hourly?stock_code=KR7005930003&day=2023-03-13&page=1
        ìê°  íì¬ê°  ë±ë½ ë±ë½ë¥ (%)  ì²´ê²°ë  ê±°ëë
0  03/13 15:30      60000     500        0.84%    1192857   11749483
1  03/13 15:15      60200     700        1.18%     103232   10545605
2  03/13 15:10      60200     700        1.18%      85065   10317828
3  03/13 15:05      60200     700        1.18%      77826   10184036
4  03/13 15:00      60200     700        1.18%      22019   10073746
*********url*** https://stock.mk.co.kr/price/hourly?stock_code=KR7005930003&day=2023-03-13&page=2
        ìê°  íì¬ê°  ë±ë½ ë±ë½ë¥ (%)  ì²´ê²°ë  ê±°ëë
0  03/13 14:30      60200     700        1.18%      38918    9419165
1  03/13 14:25      60200     700        1.18%      24045    9348829
2  03/13 14:20      60200     700        1.18%      57396    9310550
3  03/13 14:15      60100     600        1.01%      19959    9124645
4  03/13 14:10      60100     600        1.01

HTTPError: HTTP Error 500: Internal Server Error

In [10]:
# aa = pd.read_pickle(dir_name+'hyunmotor_10min_price_20230207.pkl') # 저장 내용 확인
# aa.tail()

,date,time,price,change,change(%),amount,volume
72,2023-02-07,09:20:00,170300,900,-0.53%,8987,77941
73,2023-02-07,09:15:00,170700,900,-0.52%,13959,68954
74,2023-02-07,09:10:00,170700,800,-0.47%,12741,54995
75,2023-02-07,09:05:00,170300,900,-0.53%,14507,42254
76,2023-02-07,09:00:00,171200,500,0.29%,27747,27747
